# Newsflash-Terrorism-ChartCreator

This script was developed to put a terrorist attack into a first context.

It uses two databases:

* <b>Wikipedia's list of terrorist attacks &nbsp; from 2015 up until now</b> 
* <b>The Global Terrorism Database &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; from 1970 up until last year</b> 

<br>
Both databases have their advantages and disadvantages.
While Wikipedia is constantly updated, it is nowhere near complete. A rough comparison with the Global Terrorism Database shows that Wikipedia only covers a tenth of all attacks in the timespan of 2015 to 2016 that were listed in the Global Terrorism Database. To be clear: Absolute numeric statements are not possible with Wikipedia. However, all big events seem to show up. Hence, this scraped database is a viable option for and ONLY for rankings such as:

<b>Right:</b> "This was the 5th deadliest attack of Boko Haram within the last six months"

<b>Wrong:</b>"While there were X attacks in Iraq in March, there were only Y attacks in April."

<br>

On the other end, the Global Terrorism Database (GTD) is the gold-standard in social science about terrorism.  However, it is only updated once a year with a lag of approximately six months. Also, it is for the moment discontinued due to lack of funding. Hence, it is <b>ONLY POSSIBLE</b> to make statements based on the data such as:
<br>

<b>Right:</b><i> The Iraq has seen an increase of terrorist attacks over the last 20 years </i>

<b>Wrong:</b><i> This is the Xth terrorist attack in Iraq this year.</i>

<br>
###  <b>!!! Also, <b> PLEASE BE CAREFUL WITH MIXING DATA from the scraper with the GTD</b>. Only for Western countries this is a somewhat viable option because it rarely happens that terroristic attacks happen in e.g. France and get not picked up by media and hence by Wikipedia. For countries with less media coverage this is definitely a step to avoid...</b>
<br><br>

# Pattern of use with colour code

##  <font color='red'> Red:  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </font>See the<font color='red'> preview </font> of the chart as a picture


##  <font color='orange'> Orange: </font>Decide how to<font color='orange'> filter and compare </font> by changing a code block
##  <font color='blue'>Blue: &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</font> You  <font color='blue'>create</font> the graphic by clicking the link and copy & pasting the data

# I. Getting ready

<br>
## Loading Libraries and the Global Terrorism Database

If you download the GTD from scratch, please make sure that you save the Excel File as a CSV. Otherwise the loading of the database takes forever.

In [21]:
import re
import os
import csv
import time
from vega import Vega
import requests
import datetime as dt
from datetime import datetime, timedelta
from bs4 import BeautifulSoup as BS
import pandas as pd
import numpy as np
import glob
import altair as alt
pd.options.display.max_columns = 100
GTDclean=pd.read_csv("GTDData/globalterrorismdb_0919dist.csv",low_memory=False)
GTDclean.head()

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative,alternative_txt,multiple,success,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt,corp1,target1,natlty1,natlty1_txt,targtype2,targtype2_txt,targsubtype2,targsubtype2_txt,corp2,target2,natlty2,natlty2_txt,...,weaptype2,weaptype2_txt,weapsubtype2,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3,weapsubtype3_txt,weaptype4,weaptype4_txt,weapsubtype4,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,Central America & Caribbean,National,Santo Domingo,18.456792,-69.951164,1.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0,1,0,1,Assassination,NaN,NaN,NaN,NaN,14,Private Citizens & Property,68.0,Named Civilian,NaN,Julio Guzman,58.0,Dominican Republic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,North America,Federal,Mexico city,19.371887,-99.086624,1.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0,1,0,6,Hostage Taking (Kidnapping),NaN,NaN,NaN,NaN,7,Government (Diplomatic),45.0,"Diplomatic Personnel (outside of embassy, cons...",Belgian Ambassador Daughter,"Nadine Chaval, daughter",21.0,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,NaN,Mexico,1.0,800000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,Southeast Asia,Tarlac,Unknown,15.478598,120.599741,4.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0,1,0,1,Assassination,NaN,NaN,NaN,NaN,10,Journalists & Media,54.0,Radio Journalist/Staff/Facility,Voice of America,Employee,217.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,Western Europe,Attica,Athens,37.997490,23.762728,1.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,7,Government (Diplomatic),46.0,Embassy/Consulate,NaN,U.S. Embassy,217.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Explosive,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,East Asia,Fukouka,Fukouka,33.580412,130.396361,1.0,0,NaN,NaN,1,1,1,-9,NaN,NaN,0,1,0,7,Facility/Infrastructure Attack,NaN,NaN,NaN,NaN,7,Government (Diplomatic),46.0,Embassy/Consulate,NaN,U.S. Consulate,217.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Incendiary,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


In [3]:
#renaming the columns
GTDclean=GTDclean.rename(columns={"nkill":"deaths","nwound":"injured"})
print(len(GTDclean))
#only cases without a doubt of being a terrorist attack and that fulfill all criterias of terrorism get included 
GTDclean=GTDclean[GTDclean["doubtterr"]==0]
print(len(GTDclean))
GTDclean=GTDclean[(GTDclean["crit1"]==1) & (GTDclean["crit2"]==1) & (GTDclean["crit3"]==1)]
#format dates and clean them
GTDclean["Qdate"]=GTDclean["imonth"].astype(str).str.zfill(2)+"/"+GTDclean["iday"].astype(str).str.zfill(2)+"/"+GTDclean["iyear"].astype(str)
GTDclean['Qdate'] = np.where(GTDclean['imonth']==0, '', GTDclean['Qdate'])
GTDclean['Qdate'] = np.where(GTDclean['iday']==0, '', GTDclean['Qdate'])
GTDclean["Qdate"]=pd.to_datetime(GTDclean["Qdate"])
GTDclean["year"]=GTDclean["iyear"]

#add an extracolumn to sum up incidents later
GTDclean["incidentno"]=1


#here we substract the number of dead terrorists as we do not see them as "victims"
GTDclean["deaths"]=GTDclean["deaths"].fillna(0)-GTDclean["nkillter"].fillna(0)
GTDclean["injured"]=GTDclean["injured"].fillna(0)-GTDclean["nwoundte"].fillna(0)
GTDclean["country"]=GTDclean["country_txt"]
GTDclean["region"]=GTDclean["region_txt"]

GTDclean["Qdate"]=pd.to_datetime(GTDclean["Qdate"])
GTDclean.head()


191464
146619


,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative,alternative_txt,multiple,success,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt,corp1,target1,natlty1,natlty1_txt,targtype2,targtype2_txt,targsubtype2,targsubtype2_txt,corp2,target2,natlty2,natlty2_txt,...,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3,weapsubtype3_txt,weaptype4,weaptype4_txt,weapsubtype4,weapsubtype4_txt,weapdetail,deaths,nkillus,nkillter,injured,nwoundus,nwoundte,property,propextent,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related,Qdate,year,incidentno
0,197000000001,1970,7,2,NaN,0,NaN,Dominican Republic,Dominican Republic,Central America & Caribbean,Central America & Caribbean,National,Santo Domingo,18.456792,-69.951164,1.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0,1,0,1,Assassination,NaN,NaN,NaN,NaN,14,Private Citizens & Property,68.0,Named Civilian,NaN,Julio Guzman,58.0,Dominican Republic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN,1970-07-02,1970,1
1,197000000002,1970,0,0,NaN,0,NaN,Mexico,Mexico,North America,North America,Federal,Mexico city,19.371887,-99.086624,1.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0,1,0,6,Hostage Taking (Kidnapping),NaN,NaN,NaN,NaN,7,Government (Diplomatic),45.0,"Diplomatic Personnel (outside of embassy, cons...",Belgian Ambassador Daughter,"Nadine Chaval, daughter",21.0,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,NaN,Mexico,1.0,800000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN,NaT,1970,1
2,197001000001,1970,1,0,NaN,0,NaN,Philippines,Philippines,Southeast Asia,Southeast Asia,Tarlac,Unknown,15.478598,120.599741,4.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0,1,0,1,Assassination,NaN,NaN,NaN,NaN,10,Journalists & Media,54.0,Radio Journalist/Staff/Facility,Voice of America,Employee,217.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN,NaT,1970,1
3,197001000002,1970,1,0,NaN,0,NaN,Greece,Greece,Western Europe,Western Europe,Attica,Athens,37.997490,23.762728,1.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,7,Government (Diplomatic),46.0,Embassy/Consulate,NaN,U.S. Embassy,217.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Explosive,0.0,NaN,NaN,0.0,NaN,NaN,1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN,NaT,1970,1
5,197001010002,1970,1,1,NaN,0,NaN,United States,United States,North America,North America,Illinois,Cairo,37.005105,-89.176269,1.0,0,NaN,1/1/1970: Unknown African American assailants ...,1,1,1,0,NaN,NaN,0,1,0,2,Armed Assault,NaN,NaN,NaN,NaN,3,Police,22.0,"Police Building (headquarters, station, school)",Cairo Police Department,Cairo Police Headquarters,217.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Several gunshots were fired.,0.0,0.0,0.0,0.0,0.0,0.0,1,3.0,Minor (likely < $1 million),NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"The Cairo Chief of Police, William Petersen, r...","""Police Chief Quits,"" Washington Post, January...","""Cairo Police Chief Quits; Decries Local 'Mili...","Christopher Hewitt, ""Poli

## With the next code block, the scraper gathers the Wikipedia data
## PLEASE WAIT TILL LOADED 

In [6]:
#This script covers for the years 2015 to 2018
#we check for the current year and current month 
#because we cannot pull data from Wikipedia for months and years that have not passed yet
localtime = datetime.now().strftime("%Y-%b-%d--%H-%M-%S")
current_month = datetime.now().strftime("%B")
current_year = datetime.now().strftime("%Y")

#creating a list of all months of the year that we need later
months_choices=[]
for i in range(1,13):
     months_choices.append((dt.date(2017, i, 1).strftime('%B')))
print(str(current_month)+str(current_year))
print("Wait for it...")

#this is the stopvariable that stops the scraping when we have reached the current month and year
stopvariable=0

#now we open a csv to which we write what we pull as data
with open(("ScrapedData/terror_{}.csv").format(localtime), 'w') as resultsfile:
    resultsfileWriter = csv.writer(resultsfile)
    #this is the headerrow
    firstrow=("day","month","year","attacktype","deaths","injured","location","details","attacker","context","country","link")
    resultsfileWriter.writerow(firstrow)
    
    #we add this one particular attack that has been wrongly formatted in the table
    wrongformattedrow=("8","November","2015","Suicide bombing","3","14","Ngouboua, Chad",
               "Two suicide bombers, suspected to be sent by Boko Haram, have detonated themselves in a village on the shores of Lake Chad. 3 people were killed in the blast, including two kids and another 14 were wounded.",
               "Boko Haram (suspected)","Boko Haram insurgency","Chad","https://en.wikipedia.org/wiki/List_of_terrorist_incidents_in_November_2015")
    resultsfileWriter.writerow(wrongformattedrow)

    
    #goes through the years 2015 to the current year
    for year in range(2015,(int(current_year)+1)):
        year=str(year)

        #goes through the list of months that are needed to generate the link
        for month in months_choices:
            print("...loading...")
            month=str(month)
            
            #as explained, the stopvariable is used to prevent that months and years are retrieved 
            #that have not passed yet
            if stopvariable==1:
                if month != current_month:
                        break
            
            else:
                print(month+"/"+year)
                #checks if we are at the current year and month and then sets the variable to 1
                if year == current_year:
                    if month == current_month:
                        stopvariable=1
                
                #creating the wikipedia-link from month and year defined in the for-loops
                link=("https://en.wikipedia.org/wiki/List_of_terrorist_incidents_in_"+(month)+"_"+(year))
                
                #retrieves the page with beautiful soup and the specific tables
                page = requests.get(link).text
                soup = BS(page, 'html.parser')
                #tables have different classes defined across pages, this code block
                #checks which we have
                tablesA = soup.find('table', class_="wikitable sortable")
                tablesB = soup.find('table', class_="wikitable")
                if tablesA == []:
                    tables=tablesB
                    if tablesB == []:
                        print("Both empty")
                else:
                    tables=tablesA
                    
                #retrieves all rows of the table and goes through them
                rows = tables.find_all('tr')
                for row in rows:
                    i=0
                    #retrieves for each row all cells
                    cells=(row.find_all("td"))
                    for cell in cells:
                        i+=1
                        #if it is the first cell, this is the day
                        if i==1:
                            daycell=cell.text
                            if daycell:
                                daylist=re.split("–|\+|-|\s", daycell)
                                try:
                                    type(int(daylist[0]))
                                    day=daylist[0]                                
                                except ValueError:

                                    break
                        #if it is the second cell, this is the attacktype
                        if i==2:
                            attacktype=cell.text
                            
                        #if it is the third cell, this is the number of deaths
                        #we retrieve the first number because this is the number of deaths without terrorists
                        if i==3:
                            deathcell=cell.text
                            
                            if deathcell:
                                deathlist=re.split("–|\+|-|\s|(|)", deathcell)
                                deathlist=filter(None, deathlist)
                                deathlist = [ x for x in deathlist if x.isdigit() ]
                                try:
                                    type(int(deathlist[0]))
                                    deaths=deathlist[0]
                                except (ValueError, IndexError) as e:
                                    deaths=""
                                
                        #if it is the fourth cell, this is the number of injured
                        #we retrieve the first number because this is the number of deaths without terrorists

                        if i==4:
                            injuredcell=cell.text
                            if injuredcell:

                                injuredlist=re.split("–|\+|-|\s", injuredcell)
                                injuredlist=filter(None, injuredlist)
                                injuredlist = [ x for x in injuredlist if x.isdigit() ]

                                try:
                                    type(int(injuredlist[0]))
                                    injured=injuredlist[0]
                                except (ValueError, IndexError) as e:
                                    injured=""
                                
                        #if it is the fifth cell, this is the location
                        #and normally the last string after the comma is the country

                        if i==5:
                            location=cell.text
                            if location:

                                loclist = location.split(", ")
                                country=loclist[-1]
                                country=country.strip()

                                try:
                                    locationspec=loclist[-2]
                                except IndexError:
                                    locationspec=loclist[0]

                                
                        #if it is the sixth cell, it contains further info on the attack
                        if i==6:
                            details=cell.text
                            
                        #if it is the seventh cell, it contains the name of the attacker
                        if i==7:
                            attacker=cell.text
                            if attacker:

                                attacker=attacker.replace("\n","")
                            
                        #if it is the eigth cell, it contains info on the broader conflict
                        if i==8:
                            context=cell.text
                        
                    if cells!=[]:
                        
                        #we write all this info of a row into a list and then add this list as a row to the csv
                        output_list=(day,month,year,attacktype,deaths,injured,locationspec,details,attacker,context,country, link)
                        resultsfileWriter.writerow(output_list)

#formatting certain columns to strings or to datetime                    
terror=pd.read_csv(("ScrapedData/terror_{}.csv").format(localtime))
terror["attacker"]=terror["attacker"].astype(str)
terror["country"]=terror["country"].astype(str)
terror["context"]=terror["context"].astype(str)
terror["attacktype"]=terror["attacktype"].astype(str)

terror["month"]=pd.to_datetime(terror["month"], format='%B')
terror["month"]=terror["month"].dt.month

#sorting by date and from most recent to older
terror=terror.sort_values(by=["year","month","day"])
terror=terror.reset_index(drop=True)

#creating a dateformat that our Vega to Q-code likes
terror["Qdate"]=terror["year"].astype(str)+"-"+terror["month"].astype(str).str.zfill(2)+"-"+terror["day"].astype(str).str.zfill(2)

#calculating the casualties based on deaths and injured and filling them with zeroes in case of NaN
terror["casualties"]=terror.fillna(0)['deaths']+terror.fillna(0)['injured']

#reorganising the order of the columns and then saving the data
terror=terror[['day', 'month', 'year', "Qdate" ,'attacktype',"deaths","injured","casualties","country","location","details", "attacker","context", "link"]]
terror.to_csv(("ScrapedData/terror_{}.csv").format(localtime))
terrorWiki=pd.read_csv(("ScrapedData/terror_{}.csv").format(localtime))
print("DONE")
terrorWiki.head()

October2019
Wait for it...
...loading...
January/2015


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


...loading...
February/2015
...loading...
March/2015
...loading...
April/2015
...loading...
May/2015
...loading...
June/2015
...loading...
July/2015
...loading...
August/2015
...loading...
September/2015
...loading...
October/2015
...loading...
November/2015
...loading...
December/2015
...loading...
January/2016
...loading...
February/2016
...loading...
March/2016
...loading...
April/2016
...loading...
May/2016
...loading...
June/2016
...loading...
July/2016
...loading...
August/2016
...loading...
September/2016
...loading...
October/2016
...loading...
November/2016
...loading...
December/2016
...loading...
January/2017
...loading...
February/2017
...loading...
March/2017
...loading...
April/2017
...loading...
May/2017
...loading...
June/2017
...loading...
July/2017
...loading...
August/2017
...loading...
September/2017
...loading...
October/2017
...loading...
November/2017
...loading...
December/2017
...loading...
January/2018
...loading...
February/2018
...loading...
March/2018
...lo

,Unnamed: 0,day,month,year,Qdate,attacktype,deaths,injured,casualties,country,location,details,attacker,context,link
0,0,1,1,2015,2015-01-01,Shooting\n,15.0,10.0,25.0,Cameroon,Waza Region,A group of militants attacked a crowded bus tr...,Boko Haram (suspected),Boko Haram insurgency\n,https://en.wikipedia.org/wiki/List_of_terroris...
1,1,3,1,2015,2015-01-03,Attack\n,1.0,3.0,4.0,Philippines,Maguindanao and Sultan Kudarat,An undetermined number of Bangsamoro Islamic F...,BIFF,Moro conflict\n,https://en.wikipedia.org/wiki/List_of_terroris...
2,2,3,1,2015,2015-01-03,Shooting\n,3.0,2.0,5.0,Iraq,Basra Governorate,Shooting in Basra Governorate left 5 people de...,Unknown,Iraqi Civil War\n,https://en.wikipedia.org/wiki/List_of_terroris...
3,3,3,1,2015,2015-01-03,Melee attack\n,1.0,0.0,1.0,Tunisia,El Fahs,A Tunisian policeman was ambushed by a group o...,Islamist militants,Insurgency in the Maghreb\n,https://en.wikipedia.org/wiki/List_of_terroris...
4,4,3,1,2015,2015-01-03,Massacre\n,2000.0,NaN,2000.0,Nigeria,Borno State,2015 Baga massacre: Boko Haram militants opene...,Boko Haram,Boko Haram insurgency\n,https://en.wikipedia.org/wiki/List_of_terroris...


## PLEASE WAIT TILL LOADED!!! Then let's build the graphics
# II. Building the graphics

This process is separated into four type of graphics:

##  A) Most recent context for the country
Based on your input and Wikipedia, we compare the most recent attack with previous attacks in the country of interest
##  B) Development over the years in this country
Based on the GTD, we filter by whatever definition of attacks we are interested. This could be attacks in Western Europe carried out with a bomb. Then we look how the number of deaths, injured and incidents developed over the years.
##  C)  Customized two-way and four-way group comparison
Based on the GTD, we can filter and compare groups of terrorist attacks against each other. So for instance something like: We filter down to attacks that happened in Western Europe and then compare attacks carried out with bombs, guns, cars and other items in terms of how many deaths they caused over the years and also their frequency of attacks for the latest year.

#  <font color='red'> A) Most recent context </font>
We want to create two graphics which are putting the current attack into the most recent context. For this, you have to add information about the latest attack into one of the following code block (marked <font color='orange'><b> orange</b> </font>) and also select from a list of scraped countries the one that corresponds to the one of the attack. Examples of these two graphics and the links to the templates follow right here...

##  1) Isotype Ranking

This isotype lists up the five incidents with the most deaths including the most recent event or the five incidents with the most deaths PLUS the most recent event.



<img src="Screenshots/TerrorIsotype.png" alt="Drawing" style="height: 400px;"/>

##  2) Timeline

This charts shows when the five deadliest attacks happened. It is based on loading data into NZZ's SUE-tool.



<table><tr><td><img src='Screenshots/timeline_2.png'style="height: 250px;"></td></tr></table>




#  <font color='orange'> Select the country in the second next code block to prepare the data for these charts!!! </font>

In [7]:
countrylistWiki=terrorWiki["country"].dropna().unique().tolist()
countrylistWiki.sort()
for i in countrylistWiki:
    print(i)

Abkhazia
Afghanistan
Aghanistan
Alau.Nigeria
Algeria
Angola
Antioquia
Arauquita Colombia
Argentina
Armenia
Australia
Austria
Baghdad Iraq
Bahrain
Bangkok
Bangladesh
Belarus
Belfast Northern Ireland
Belgium
Bolivia
Border Region Niger
Bosnia and Herzegovina
Brazil
Burkina Faso
Burundi
Cameroon
Canada
Central African Republic
Chad
Chile
China
Colombia
Crimea
Dagestan
Damascus Syria
Darrang district,India
Democratic Republic of Congo
Democratic Republic of The Congo
Democratic Republic of the Congo
Denmark
Dih Bala district,Afghanistan
East Jerusalem
East Jerusalem,
Ecuador
Egypt
Egypt.
El-Baraf Somalia
Ethiopia
Finland
France
Gaza Strip
Georgia)
Germany
Gilgit-Baltistan
Greece
Guatemala
Haiti
Hebron
Hungary
Idlib,Syria
India
Indonesia
Iran
Iraq
Ireland
Israel
Italia
Italy
Ivory Coast
Japan
Jerusalem
Jilib,Somalia
Jordan
Kazakhstan
Keana Nigeria
Kenya
Khost Province
Kibirizi Democratic Republic of Congo
Kosovo
Kuwait
Kyrgyzstan
La Araucania Region and Bio-Bio Region
Lahj Governorate Yemen

By selecting one or more of the countries listed from the earlier code block and then pasting the countryname into the following code block, we define which country/which countries we want to look at
<br>
<br>

<i>countryfilterWiki=["Afghanistan"]</i>
OR
<i>countryfilterWiki=["Afghanistan, Iraq"]</i>
<br>

In [8]:
countryfilterWiki=["France"]


#  <font color='orange'> Additionally, you have to define in the next code block: </font>
* How many days we want to look back. Per default, the value is 365, so an entire year
* The German name of the country that you are interested in
* The details of the attack that has just happened

In [9]:
daystogoback=365

add_year=2019
add_month=10
add_dayofthemonth=9

add_deaths=2
add_injured=0

add_country="Germany"
add_city="X"


##  Now let the code do the rest

In [20]:
#we calculate what is the oldest date to include
add_details="Current attack"
add_attacktype="Current attack"
add_attacker="Current attack"
add_context="Current attack"
timeback=(datetime.now())- timedelta(days=daystogoback)
#print(timeback)
terror["Qdate"]=terror["Qdate"].astype(str).astype('datetime64[ns]')
subframe=terror[terror["Qdate"]>timeback]
subframe.reset_index(drop=True)

#we filter for countries included in our filter bracket
if countryfilterWiki[0]!="XYZ":
    subframe=subframe[subframe["country"].isin(countryfilterWiki)]
    
#we sort the dataframe by date
subframe=subframe.sort_values(by="Qdate", ascending=False)
terrorWiki_red=subframe.reset_index(drop=True)

#and set for all entries from Wikipedia the column "most recent" to zero for false
addedframe=terrorWiki_red
addedframe["mostrecent"]=0

#then we create a row for our dataframe based on our own inputs
if add_dayofthemonth != 0:
    extraQdate=str(add_year)+"-"+str(add_month).zfill(2)+"-"+str(add_dayofthemonth).zfill(2)
    add_extraQdate=datetime.strptime(extraQdate, '%Y-%m-%d')
    add_location=str(add_city)
    add_casualties=add_deaths+add_injured
    add_link="This is our incident"
    extraarray=[add_dayofthemonth,add_month,add_year, add_extraQdate, add_attacktype,add_deaths,add_injured,add_casualties,add_country,add_location,add_details,add_attacker,add_context,add_link,1]
    
    #and add it at the bottom
    addedframe.loc[-1] = extraarray  # adding a row
    addedframe.index = addedframe.index + 1  # shifting index
    addedframe = addedframe.sort_index()  # sorting by index
addedframe["Qdate"]=pd.to_datetime(addedframe["Qdate"])

#then we sort by deaths and take the top 5 
sortcriteria="deaths"
topx=5
sortedframe=addedframe.sort_values(by=sortcriteria, ascending=False)
sortedframe=sortedframe.reset_index(drop=True)
rankedframe=sortedframe.head(topx)

#if in these top 5 our most recent attack is included, we write this head dataframe out as a csv streight
if any(rankedframe.mostrecent == 1):
    rankedframe.to_csv(("FilteredData/scraper_recentattack_intop{}_incidents_sortedby_{}.csv").format(topx,sortcriteria), index=False)
    chronframe=rankedframe.sort_values(by="Qdate", ascending=False)
    chronframe.to_csv(("FilteredData/scraper_recentattack_intop{}_incidents_sortedby_date.csv").format(topx), index=False)

#Otherwise we add the most recent event at the end of the dataframe and then write the result out
else:
    mostrecentevent=sortedframe[sortedframe["mostrecent"]==1]
    rankedframe=pd.concat([rankedframe,mostrecentevent], sort=False)
    rankedframe.to_csv(("FilteredData/scraper_recentattack_vs_top{}_incidents_sortedby_{}.csv").format(topx,sortcriteria), index=False)
    chronframe=rankedframe.sort_values(by="Qdate", ascending=False)
    chronframe.to_csv(("FilteredData/scraper_recentattack_vs_top{}_incidents_sortedby_date.csv").format(topx), index=False)
import datetime as dt
#Filtering for only the most recent incident and reducing the dataframe to date, criteria and location
mostrecentdf=rankedframe[rankedframe["mostrecent"]==1]
mostrecentdf=mostrecentdf[["Qdate",sortcriteria,"location"]]

#renaming one column so that we have a column for recent_deaths here and
#deaths in the other dataframe
newcolumname="recent_"+str(sortcriteria)
mostrecentdf=mostrecentdf.rename(columns={sortcriteria: newcolumname})

#Filtering for all other cases and sorting them by our criterion
contextdf=rankedframe[rankedframe["mostrecent"]==0]
contextdf=contextdf[["Qdate",sortcriteria,"location"]]
contextdf=contextdf.sort_values(by=sortcriteria, ascending=False)

#merging these two dataframes so that we start with the most recent attack and then the other attacks
#by decreasing number of fatalities or other criterion
isotypedf=pd.concat([mostrecentdf,contextdf], sort=False)
isotypedf=isotypedf[["Qdate","location",sortcriteria,newcolumname]]
isotypedf=isotypedf.reset_index(drop=True)

#we format the date and then create the "label" column from date and location
isotypedf['Qdate']=isotypedf['Qdate'].astype(str).astype('datetime64[ns]').dt.strftime(' (am %d.%m.%Y)')
isotypedf["Label"]=isotypedf['location'].astype(str) + isotypedf['Qdate']

#then we resort our isotype dataframe with just three columns and fill emtpy cells with zero
isotypedf=isotypedf[["Label","deaths","recent_deaths"]]
isotypedf["deaths"]=isotypedf["deaths"].fillna(0).astype('int64')
isotypedf["recent_deaths"]=isotypedf["recent_deaths"].fillna(0).astype('int64')
#we translate the columns to German and then write out the csv
isotypedf=isotypedf.rename(index=str, columns={"deaths": "Tote früherer Anschläge", "recent_deaths": "Tote aktueller Anschlag"})
isotypedf.to_csv("chartdata/1isotype.csv", index=False)

#also we write a txt file with a default description on what is shown in the chart
top5check=(len(isotypedf))
if top5check>5:
    text_file = open("chartdata/1isotype.txt", "w")

    text_file.write("Untertitel: \n")
    text_file.write("Der aktuelle Anschlag im Vergleich zu den fünf tödlichsten Anschläge der letzten {} Tage".format(daystogoback))
    text_file.close()
else:
    text_file = open("chartdata/1isotype.txt", "w")

    text_file.write("Untertitel: \n")
    text_file.write("Der aktuelle Anschlag gehört zu den fünf tödlichsten Anschläge der letzten {} Tage".format(daystogoback))
    text_file.close()

#now we build the csv for ouur timeline by again building a dataframe for the recent attack and all other attacks
mostrecentdf=rankedframe[rankedframe["mostrecent"]==1].copy()
mostrecentdf["mostrecent"]="Aktuelle Attacke"
mostrecentdf=mostrecentdf[["Qdate","deaths","location","mostrecent"]]

contextdf=rankedframe[rankedframe["mostrecent"]==0].copy()
contextdf["mostrecent"]="Vorherige Attacken"
contextdf=contextdf[["Qdate","deaths","location","mostrecent"]]
contextdf=contextdf.sort_values(by="Qdate",ascending=False)

#then we start a dataframe filled with our start date (how far we went back) in the first row
timebackdate=add_extraQdate - timedelta(days=daystogoback)
startdf=pd.DataFrame(data=[[timebackdate,0,"","Startdatum"]],columns=["Qdate","deaths","location","mostrecent"])

#then we further add put together a dataframe of the startdate, the recent attack and the last big attacks
timeline=pd.concat([mostrecentdf,contextdf,startdf], sort=False)
#and add two columns for layouting the labels to prevent overlap
timeline["DesktopOrder"]=1
timeline["MobileOrder"]=0
#again we fill the Nan with zeroes
timeline["deaths"]=timeline["deaths"].fillna(0).astype('int64')

timeline['Qdate']=timeline['Qdate'].astype(str).astype('datetime64[ns]').dt.strftime('%d.%m.%Y')


timeline['Enddatum']=""
#we reorder the columns and rename them and then write a csv
timeline=timeline[["Qdate","Enddatum","mostrecent","location","deaths"]]
timeline=timeline.rename(index=str, columns={"Qdate":"Datum","deaths": "Anzahl Opfer", "mostrecent": "Kategorie","location":"Beschreibung"})
timeline.to_csv("chartdata/2timeline.csv", index=False)

#depending on whether the recent attack is in the top 5 or falls out of it, our txt-default description is different
if top5check>5:
    text_file = open("chartdata/2timeline.txt", "w")

    text_file.write("Untertitel: \n")
    text_file.write("Der aktuelle Anschlag im zeitlichen Kontext zu den fünf tödlichsten Anschlägen der letzten {} Tage".format(daystogoback))
    text_file.close()
else:
    text_file = open("chartdata/2timeline.txt", "w")
    text_file.write("Untertitel: \n")
    text_file.write("Der aktuelle Anschlag ist der jüngste der fünf tödlichsten Anschläge der letzten {} Tage".format(daystogoback))
    text_file.close()
subframe.to_csv("FilteredData/filtered_wikidataset.csv",index=False)
print("DONE")


DONE


# <font color='blue'> Links to the charts </font>

## 1) Isotype

This isotype lists up the five incidents with the most deaths including the most recent event or the five incidents with the most deaths PLUS the most recent event.


[This is the link to the chart template](https://q.st.nzz.ch/item/e36abb8b21ffff55d3a441215e45d6e2)

<img src="Screenshots/TerrorIsotype.png" alt="Drawing" style="height: 400px;"/>


## 2) Timeline

This charts shows when the five deadliest attacks happened. 

[This is the link to the sue chart template](https://sue.st.nzz.ch/create/timeline)


<table><tr><td><img src='Screenshots/timeline_2.png'style="height: 250px;"></td></tr></table>



# <font color='red'>B) Development over the years</font> 
Here we want to filter down by whatever category we are interested in tp see what was the development over the years. <br>
The four different possible filters are: Region, Country, Weapontype, Targettype. 
<br> So for instance, you can filter by number of deaths in Afghanistan. Or attacks where a car was used in Western Europe. For this, you need to select from the list-ups the entries that you are interested in and put them in the following code bracket (marked <font color='orange'> orange </font>) like:
<br>
<br>
<i>countryfilterGTD=["Afghanistan"]</i>
<br>OR<br>
<i>regionfilterGTD=["Western Europe","Eastern Europe"]</i>
<br>
<br>
If you do not need the particular filter, just make sure the first value in the bracket is "XYZ" like:
<br>
<br>
<i>countryfilterGTD=["XYZ"]</i>
<br>or<br>
<i>regionfilterGTD=["XYZ","XYZ"]</i>

Examples of these two graphics and the links to the template follow right here...

## &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 3) Stacked Bars along X-Axis as Time

The stacked bar-chart timeline shows how the number of deaths and injured has changed over the years for the cases that we are interested in. 


<img src="Screenshots/StackedBar.png" alt="Drawing" style="height: 400px;"/>

## &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 4)  Bars along X-Axis as Time

The bar-chart timeline shows the number of attacks over the years for the cases that we are interested in.

<img src="Screenshots/IncidentsProYear.png" alt="Drawing" style="height: 400px;"/>



#  <font color='orange'> Select the country in the second next code block here!!! </font>


In [26]:
countryrlistGTD=GTDclean["country"].unique().tolist()
countryrlistGTD.sort()
for i in countryrlistGTD:
    print(i)

Afghanistan
Albania
Algeria
Andorra
Angola
Argentina
Armenia
Australia
Austria
Azerbaijan
Bahamas
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bhutan
Bolivia
Bosnia-Herzegovina
Botswana
Brazil
Brunei
Bulgaria
Burkina Faso
Burundi
Cambodia
Cameroon
Canada
Central African Republic
Chad
Chile
China
Colombia
Comoros
Costa Rica
Croatia
Cuba
Cyprus
Czech Republic
Czechoslovakia
Democratic Republic of the Congo
Denmark
Djibouti
Dominica
Dominican Republic
East Germany (GDR)
East Timor
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Ethiopia
Falkland Islands
Fiji
Finland
France
French Guiana
French Polynesia
Gabon
Gambia
Georgia
Germany
Ghana
Greece
Grenada
Guadeloupe
Guatemala
Guinea
Guinea-Bissau
Guyana
Haiti
Honduras
Hong Kong
Hungary
Iceland
India
Indonesia
International
Iran
Iraq
Ireland
Israel
Italy
Ivory Coast
Jamaica
Japan
Jordan
Kazakhstan
Kenya
Kosovo
Kuwait
Kyrgyzstan
Laos
Latvia
Lebanon
Lesotho
Liberia
Libya
Lithuania
Luxembourg
Macau
Macedonia
Madagascar
Ma

In [11]:
countryfilterGTD=["France"]

#  <font color='orange'> Select the region in the second next code block here!!! </font>


In [12]:
regionlist=GTDclean["region"].unique().tolist()
regionlist.sort()
for i in regionlist:
    print(i)

Australasia & Oceania
Central America & Caribbean
Central Asia
East Asia
Eastern Europe
Middle East & North Africa
North America
South America
South Asia
Southeast Asia
Sub-Saharan Africa
Western Europe


In [13]:
regionfilterGTD=["XYZ"]

#  <font color='orange'> Select the weapontype in the second next code block here!! </font>


In [14]:
weaptypelist=GTDclean["weaptype1_txt"].unique().tolist()
weaptypelist.sort()
for i in weaptypelist:
    print(i)

Biological
Chemical
Explosives
Fake Weapons
Firearms
Incendiary
Melee
Other
Radiological
Sabotage Equipment
Unknown
Vehicle (not to include vehicle-borne explosives, i.e., car or truck bombs)


In [15]:
weaptypefilterGTD=["XYZ"]

#  <font color='orange'> Select the target type in the second next code block here!! </font>


In [16]:
targettypelist=GTDclean["targtype1_txt"].unique().tolist()
targettypelist.sort()
for i in targettypelist:
    print(i)

Abortion Related
Airports & Aircraft
Business
Educational Institution
Food or Water Supply
Government (Diplomatic)
Government (General)
Journalists & Media
Maritime
Military
NGO
Other
Police
Private Citizens & Property
Religious Figures/Institutions
Telecommunication
Terrorists/Non-State Militia
Tourists
Transportation
Unknown
Utilities
Violent Political Party


In [17]:
targtypefilterGTD=["XYZ"]

# Let the code do the rest

In [19]:
subframe=GTDclean
criteria_txt_array=[]
criteria_array=[]

#whenever the first entry of the filterlist/bracket is not XYZ,
#we use this list and only take entries into the sunframe that fulfill the filtercritera
if countryfilterGTD[0]!="XYZ":
    subframe=subframe[subframe["country"].isin(countryfilterGTD)]
    sub_array=[]
    sub_array.append("for the country(ies):"+str(countryfilterGTD))
    criteria_txt_array.append(sub_array)
    criteria_array.extend(countryfilterGTD)
    
if regionfilterGTD[0]!="XYZ":
    subframe=subframe[subframe["region"].isin(regionfilterGTD)]
    sub_array=[]
    sub_array.append("for the region(s):"+str(regionfilterGTD))
    criteria_txt_array.append(sub_array)
    criteria_array.extend(regionfilterGTD)

    
if weaptypefilterGTD[0]!="XYZ":
    subframe=subframe[(subframe["weaptype1_txt"].isin(weaptypefilterGTD)) | (subframe["weaptype2_txt"].isin(weaptypefilterGTD))| (subframe["weaptype3_txt"].isin(weaptypefilterGTD))]
    sub_array=[]
    sub_array.append("for the weapontype(s):"+str(weaptypefilterGTD))
    criteria_txt_array.append(sub_array)
    criteria_array.extend(weaptypefilterGTD)

if targtypefilterGTD[0]!="XYZ":
    subframe=subframe[(subframe["targtype1_txt"].isin(targtypefilterGTD)) | (subframe["targtype2_txt"].isin(targtypefilterGTD))| (subframe["targtype3_txt"].isin(targtypefilterGTD))]
    sub_array=[]
    sub_array.append("for the targettype(s):"+str(targtypefilterGTD))
    criteria_txt_array.append(sub_array)
    criteria_array.extend(targtypefilterGTD)
criteria_txt_array=str(criteria_txt_array).replace("[","").replace("]","")
print(criteria_txt_array)

#then we sort all the values by date
subframe=subframe.sort_values(by="Qdate", ascending=False)
terrorGTD=subframe.reset_index(drop=True)
terrorGTD.head()

#now we build the dataframe for our stackedbars by only including year, deaths and injured and then sum over years
stackedbars=terrorGTD[["year","deaths","injured"]]
stackedbars=stackedbars.groupby("year").sum()
stackedbars=stackedbars.reset_index()
for year in range(1970, 2017):
#if we have no entry for one year between 1970 and 2017, we create one with two zeroes
    if (stackedbars["year"]==year).any():
        pass
    else:
        stackedbars=stackedbars.append({"year":year,"deaths":0,"injured":0}, ignore_index=True)
        
#sort the dataframe, rename the columns and save the result as a csv
stackedbars=stackedbars.sort_values(by="year")   
stackedbars=stackedbars.reset_index(drop=True)
stackedbars=stackedbars.rename(index=str, columns={"year": "Jahr", "deaths": "Tote","injured":"Verletzte"})
stackedbars.to_csv("chartdata/3stackedbars.csv", index=False)
text_file = open("chartdata/3stackedbars.txt", "w")
text_file.write("Untertitel: \n")
text_file.write("Die Zahl der Toten und Verletzten durch Terroranschläge in {} über die Jahre (keine Daten für 1993)".format(criteria_txt_array))
text_file.close()

#similar procedure for the barchart, just with incidents
barchart=terrorGTD[["year","incidentno"]]
barchart=barchart.groupby("year").sum()
barchart=barchart.reset_index()
for year in range(1970, 2017):
    if (barchart["year"]==year).any():
        pass
    else:
        barchart=barchart.append({"year":year,"incidentno":0}, ignore_index=True)
barchart=barchart.sort_values(by="year")   
barchart=barchart.reset_index(drop=True)
barchart=barchart.rename(index=str, columns={"year": "Jahr", "incidentno": "Zahl der Anschläge"})
barchart.to_csv("chartdata/4barchart.csv", index=False)
text_file = open("chartdata/4barchart.txt", "w")
text_file.write("Untertitel: \n")
text_file.write("Die Zahl der Terroranschläge in {} über die Jahre (keine Daten für 1993)".format(criteria_txt_array))
text_file.close()
subframe.to_csv("FilteredData/filtered_GTD_subframe.csv")
print("DONE")

"for the country(ies):'France'"
DONE


# <font color='blue'> Links to the charts </font>

## 3) Stacked Bars along X-Axis as Time

The stacked bar-chart timeline shows how the number of deaths and injured has changed over the years for the cases that we are interested in. 

[This is the link to the chart template](https://q.st.nzz.ch/item/e36abb8b21ffff55d3a441215e1bd190)

<img src="Screenshots/StackedBar.png" alt="Drawing" style="height: 400px;"/>


## 4) Bars along X-Axis as Time

The bar-chart timeline shows the number of attacks over the years for the cases that we are interested in.

[This is the link to the chart template](https://q.st.nzz.ch/item/2db271024d5db0b45dcf41d7cf602ec8)
<img src="Screenshots/IncidentsProYear.png" alt="Drawing" style="height: 400px;"/>



# <font color='red'> C) Default two-way and four-way comparison </font>
Here we want to make a comparison that has certain default mechanisms included and is optimised for country comparisons. For this, you need to select from the following country list-up  one country that you are interested in and paste it into the following code bracket (marked <font color='red'> red </font>) and also the German name for the country like:
<br>
<br>
<i>countryofinterest=["Albania"]</i>
<br>
<i>German_country_Name=["Albanien"]</i>

<br>
<br>
Examples of the two graphics that will use this data and the links to the template follow right here...

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 5) Two-Way Lines / Grouped Bar Chart /Small Multiples (Default)
Here, the data is prepared for a comparison between the country of interest and the country in the region with the highest number of deaths. The same CSV can be used for 

<table><tr><td><img src='Screenshots/TwoCountryLineChart.png'></td></tr>
</table>


#  <font color='orange'> Select the country in the second next code block here!!! </font>


In [35]:
countryrlistGTD=GTDclean["country"].unique().tolist()
countryrlistGTD.sort()
for i in countryrlistGTD:
    print(i)

Afghanistan
Albania
Algeria
Andorra
Angola
Argentina
Armenia
Australia
Austria
Azerbaijan
Bahamas
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bhutan
Bolivia
Bosnia-Herzegovina
Botswana
Brazil
Brunei
Bulgaria
Burkina Faso
Burundi
Cambodia
Cameroon
Canada
Central African Republic
Chad
Chile
China
Colombia
Comoros
Costa Rica
Croatia
Cuba
Cyprus
Czech Republic
Czechoslovakia
Democratic Republic of the Congo
Denmark
Djibouti
Dominica
Dominican Republic
East Germany (GDR)
East Timor
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Ethiopia
Falkland Islands
Fiji
Finland
France
French Guiana
French Polynesia
Gabon
Gambia
Georgia
Germany
Ghana
Greece
Grenada
Guadeloupe
Guatemala
Guinea
Guinea-Bissau
Guyana
Haiti
Honduras
Hong Kong
Hungary
Iceland
India
Indonesia
International
Iran
Iraq
Ireland
Israel
Italy
Ivory Coast
Jamaica
Japan
Jordan
Kazakhstan
Kenya
Kosovo
Kuwait
Kyrgyzstan
Laos
Latvia
Lebanon
Lesotho
Liberia
Libya
Lithuania
Luxembourg
Macau
Macedonia
Madagascar
Ma

In [36]:
countryofinterest=["Germany"]
German_country_Name=["Deutschland"]

## And now let the code do the rest

In [37]:
countryofinterest=countryofinterest[0]
#we calculate the casualties
GTDclean["casualties"]=GTDclean["deaths"]+GTDclean["injured"]
#then we sum by country region and year
smallmultiplesdfREG=GTDclean.groupby(["country","region","year"]).sum().reset_index()
#and from these yearly values, we calculate for each country a median
medianpercountry=smallmultiplesdfREG.groupby(["country","region"]).median().reset_index()
medianpercountry=medianpercountry[["country","region","year","deaths","injured","casualties"]]
#now we extract the region in which our country of interested is located
compareregion=(medianpercountry[medianpercountry["country"]==countryofinterest]["region"]).iloc[0]
#and get the values for all other countries that are in this region and but not the country of interest
regionalcomparison=smallmultiplesdfREG[(smallmultiplesdfREG["region"]==compareregion) & (smallmultiplesdfREG["country"]!=countryofinterest)]
#then we look for the country in this region with the worst median number of deaths
regionalcomparison=regionalcomparison.sort_values(by="deaths", ascending=False)
worstcasecomp=regionalcomparison["country"].iloc[0]

#then we create two dataframes. one with the country of interest and one with the worst case comparison of the region
countrydf=smallmultiplesdfREG[smallmultiplesdfREG["country"]==countryofinterest]
countrydf=countrydf.sort_values(by="year").reset_index(drop=True).rename(columns={"deaths":countryofinterest})
for year in range(1970, 2017):
#if we have no entry for one year between 1970 and 2017, we create one with two zeroes
    if (countrydf["year"]==year).any():
        pass
    else:
        countrydf=countrydf.append({"country":countryofinterest,"region":compareregion,"year":year,"deaths":0,"injured":0,"casualties":0}, ignore_index=True)
countrydf=countrydf.sort_values(by="year").reset_index(drop=True)

comparedcountrydf=smallmultiplesdfREG[smallmultiplesdfREG["country"]==worstcasecomp]
comparedcountrydf=comparedcountrydf.sort_values(by="year").reset_index(drop=True).rename(columns={"deaths":worstcasecomp})
for year in range(1970, 2017):
#if we have no entry for one year between 1970 and 2017, we create one with two zeroes
    if (comparedcountrydf["year"]==year).any():
        pass
    else:
        comparedcountrydf=comparedcountrydf.append({"country":worstcasecomp,"region":compareregion,"year":year,"deaths":0,"injured":0,"casualties":0}, ignore_index=True)
comparedcountrydf=comparedcountrydf.sort_values(by="year").reset_index(drop=True)
        



#then we merge these two dataframes and put out this as the default country comparison
smallmultipledf1on1=pd.concat([countrydf[["year",countryofinterest]],comparedcountrydf[worstcasecomp]],axis=1, sort=False)
smallmultipledf1on1=smallmultipledf1on1.fillna(0)
smallmultipledf1on1=smallmultipledf1on1.rename(index=str, columns={"year": "Jahr", "countryofinterest": "German_country_Name"})
smallmultipledf1on1.to_csv("ChartData/5TwoCountriesComparison_default.csv", index=False)
text_file = open("chartdata/5TwoCountriesComparison_default.txt", "w")
text_file.write("Untertitel: \n")
text_file.write("Die jährliche Zahl der Anschlagstoten in {0} und {1} im Vergleich".format(German_country_Name, worstcasecomp))
text_file.close()

GTDclean=GTDclean.sort_values(by="year", ascending=False)
GTDclean

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative,alternative_txt,multiple,success,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt,corp1,target1,natlty1,natlty1_txt,targtype2,targtype2_txt,targsubtype2,targsubtype2_txt,corp2,target2,natlty2,natlty2_txt,...,weaptype3,weaptype3_txt,weapsubtype3,weapsubtype3_txt,weaptype4,weaptype4_txt,weapsubtype4,weapsubtype4_txt,weapdetail,deaths,nkillus,nkillter,injured,nwoundus,nwoundte,property,propextent,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related,Qdate,year,incidentno,casualties
191462,201812310033,2018,12,31,NaN,0,NaN,Afghanistan,Afghanistan,South Asia,South Asia,Helmand,Nad Ali district,31.639085,64.243436,3.0,0,The incident occurred in the Naqilabad area.,12/31/2018: Assailants attacked a police outpo...,1,1,1,0,NaN,NaN,0,1,0,9,Unknown,NaN,NaN,NaN,NaN,3,Police,22.0,"Police Building (headquarters, station, school)",Afghan Police,Outpost,4.0,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,3.0,0.0,0.0,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Afghan War Casualty Report: Dec. 28-Jan. 3,"" ...",NaN,NaN,START Primary Collection,0,0,0,0,NaN,2018-12-31,2018,1,4.0
185042,201805010016,2018,5,1,NaN,0,NaN,Nigeria,Nigeria,Sub-Saharan Africa,Sub-Saharan Africa,Benue,Tse Iortyer,7.954114,8.748117,2.0,0,The incident occurred in Guma district,"05/01/2018: Assailants attacked Tse Iortyer, G...",1,1,1,0,NaN,NaN,0,1,0,2,Armed Assault,7.0,Facility/Infrastructure Attack,NaN,NaN,14,Private Citizens & Property,75.0,Village/City/Town/Suburb,Tse Iortyer Village,Village,147.0,Nigeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,0.0,NaN,16.0,0.0,NaN,1,3.0,Minor (likely < $1 million),-99.0,Houses were damaged in this attack.,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Casualty numbers conflict across sources. Foll...,"""Suspected herdsmen kill six in Benue communit...","""BREAKING: 10 killed as herdsmen attack Guma i...","""10 killed as herdsmen attack Guma in Benue,"" ...",START Primary Collection,-9,-9,0,-9,NaN,2018-05-01,2018,1,26.0
185030,201804300031,2018,4,30,NaN,0,NaN,Philippines,Philippines,Southeast Asia,Southeast Asia,Negros Oriental,Dumaguete,9.317282,123.302489,1.0,0,The incident occurred along Aldecoa road in th...,04/30/2018: Assailants opened fire on Edmund S...,1,1,1,0,NaN,NaN,0,1,0,1,Assassination,NaN,NaN,NaN,NaN,10,Journalists & Media,54.0,Radio Journalist/Staff/Facility,yGB 91.7 FM,Radio Broadcaster: Edmund Sestoso,160.0,Philippines,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""PTFoMS seeks help for info on killers of Duma...","""NPA condemns broadcaster’s slay,"" The Daily G...","""ASD -- Force Protection/Situational Awareness...",START Primary Collection,0,0,0,0,NaN,2018-04-30,2018,1,1.0
185031,201804300032,2018,4,30,NaN,1,5/2/18,Cameroon,Cameroon,Sub-Saharan Africa,Sub-Saharan Africa,North-West,Bamenda,5.963716,10.159570,1.0,1,NaN,"04/30/2018: Assailants abducted two priests, i...",1,1,1,0,NaN,NaN,0,1,0,6,Hostage Taking (Kidnapping),NaN,NaN,NaN,NaN,8,Educational Institution,50.0,Other Personnel,Saint Bede's College,Principal: Father Willian Niba,37.0,Cameroon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,1.0,2.0,0.0,NaN,2.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,N

# <font color='blue'> Links to the charts </font>

## 5) Two-Way Lines / Grouped Bar Chart /Small Multiples (Default)
Here, the data is prepared for a comparison between the country of interest and the country in the region with the highest number of deaths. The same CSV can be used for 
* 5a) [Lines Comparison Template](https://q.st.nzz.ch/item/1a9e160a22e9c34584cf45c8d561779b)

<table><tr><td><img src='Screenshots/TwoCountryLineChart.png'></td></tr>
</table>

# <font color='red'> D) Custom 2-way comparison </font>
Here we want to be able to compare two or four categories against each other. For this we have to make two decisions:
1. FILTER: What we want to filter down by, so for instance only attacks in Afghanistan
2. COMPARE: What we want to compare? So for example different types of weapons used in the attacks in Afghanistan.
<br>
<b>The four categories you can use for the filter and the comaprison are: region, country, weapontype and target of attack</b>
<br>

## 1-FILTER
To filter down by whatever category we are interested in and see what was the development over the years. So, number of deaths in Afghanistan. Or attacks where a car was used in Western Europe. For this, you need to select from the following list-ups the one that you are interested in and put them again in the following code bracket (marked <font color='red'> red </font>) like:
<br>
<br>
<i>countryfilterGTD=["Afghanistan"]</i>
<br>OR<br>
<i>regionfilterGTD=["Western Europe","Eastern Europe"]</i>
<br>
<br>
If you do not need the particular filter, just make sure the first value in the bracket is "XYZ" like:
<br>
<br>
<i>countryfilterGTD=["XYZ"]</i>
<br>or<br>
<i>regionfilterGTD=["XYZ","XYZ"]</i>

## 2-COMPARISON
If you do not want to filter by this category but actually use it for the comparison, just add in the same codebracket the two or four entitires that you want to compare against each other, so something like
<i>countrytwowaycomparison=["Afghanistan","Iraq"]</i>
<i>countryfourwaycomparison=["Afghanistan","Iraq","Germany","France"]</i>
<br>
<br>

## PLEASE KEEP IN MIND THAT:
a) You cannot filter and compare by the same category <br>
b) You can only compare by one category

Examples of these two graphics and the links to the template follow right here...


## &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 6) Two-Way Lines (Custom)
Here, the data is prepared for a comparison between the two groups of interests. The same CSV can be used for 

<table><tr><td><img src='Screenshots/DoubleLineCustom_2.png' style="height: 400px;"></td></tr>
</table>


## Here are the list ups...

#  <font color='orange'> Select the country here!!! </font>


In [38]:
countryrlistGTD=GTDclean["country"].unique().tolist()
countryrlistGTD.sort()
for i in countryrlistGTD:
    print(i)

Afghanistan
Albania
Algeria
Andorra
Angola
Argentina
Armenia
Australia
Austria
Azerbaijan
Bahamas
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bhutan
Bolivia
Bosnia-Herzegovina
Botswana
Brazil
Brunei
Bulgaria
Burkina Faso
Burundi
Cambodia
Cameroon
Canada
Central African Republic
Chad
Chile
China
Colombia
Comoros
Costa Rica
Croatia
Cuba
Cyprus
Czech Republic
Czechoslovakia
Democratic Republic of the Congo
Denmark
Djibouti
Dominica
Dominican Republic
East Germany (GDR)
East Timor
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Ethiopia
Falkland Islands
Fiji
Finland
France
French Guiana
French Polynesia
Gabon
Gambia
Georgia
Germany
Ghana
Greece
Grenada
Guadeloupe
Guatemala
Guinea
Guinea-Bissau
Guyana
Haiti
Honduras
Hong Kong
Hungary
Iceland
India
Indonesia
International
Iran
Iraq
Ireland
Israel
Italy
Ivory Coast
Jamaica
Japan
Jordan
Kazakhstan
Kenya
Kosovo
Kuwait
Kyrgyzstan
Laos
Latvia
Lebanon
Lesotho
Liberia
Libya
Lithuania
Luxembourg
Macau
Macedonia
Madagascar
Ma

In [39]:
countryfilterGTD=["XYZ"]
countryGTD_twowaycomparison=["Afghanistan","Germany"]
countryGTD_twowaycomparison_German=["Afghanistan","Deutschland"]


#  <font color='orange'> Select the region here!!! </font>


In [40]:
regionlist=GTDclean["region"].unique().tolist()
regionlist.sort()
for i in regionlist:
    print(i)

Australasia & Oceania
Central America & Caribbean
Central Asia
East Asia
Eastern Europe
Middle East & North Africa
North America
South America
South Asia
Southeast Asia
Sub-Saharan Africa
Western Europe


In [41]:
regionfilterGTD=["XYZ"]
 
regionGTD_twowaycomparison=["XYZ","XYZ"]
regionGTD_twowaycomparison_German=["XYZ","XYZ"]


#  <font color='orange'> Select the weapontype here!!! </font>


In [42]:
weaptypelist=GTDclean["weaptype1_txt"].unique().tolist()
weaptypelist.sort()
for i in weaptypelist:
    print(i)

Biological
Chemical
Explosives
Fake Weapons
Firearms
Incendiary
Melee
Other
Radiological
Sabotage Equipment
Unknown
Vehicle (not to include vehicle-borne explosives, i.e., car or truck bombs)


In [43]:
weaptypefilterGTD=["Firearms"]

weaptypeGTD_twowaycomparison=["XYZ","XYZ"]
weaptypeGTD_twowaycomparison_German=["XYZ","XYZ"]


#  <font color='orange'> Select the target type here!!! </font>


In [44]:
targettypelist=GTDclean["targtype1_txt"].unique().tolist()
targettypelist.sort()
for i in targettypelist:
    print(i)

Abortion Related
Airports & Aircraft
Business
Educational Institution
Food or Water Supply
Government (Diplomatic)
Government (General)
Journalists & Media
Maritime
Military
NGO
Other
Police
Private Citizens & Property
Religious Figures/Institutions
Telecommunication
Terrorists/Non-State Militia
Tourists
Transportation
Unknown
Utilities
Violent Political Party


In [45]:
targtypefilterGTD=["XYZ"]

targtypeGTD_twowaycomparison=["XYZ","XYZ"]
targtypeGTD_twowaycomparison_German=["XYZ","XYZ"]

# Let the code do the rest

In [47]:
subframe=GTDclean
criteria_txt_array=[]
criteria_array=[]

#again we filter for the criteria that we set in the filterlists
if countryfilterGTD[0]!="XYZ":
    subframe=subframe[subframe["country"].isin(countryfilterGTD)]
    sub_array=[]
    sub_array.append("for the country(ies):"+str(countryfilterGTD))
    criteria_txt_array.append(sub_array)
    criteria_array.extend(countryfilterGTD)
    
if regionfilterGTD[0]!="XYZ":
    subframe=subframe[subframe["region"].isin(regionfilterGTD)]
    sub_array=[]
    sub_array.append("for the region(s):"+str(regionfilterGTD))
    criteria_txt_array.append(sub_array)
    criteria_array.extend(regionfilterGTD)

    
if weaptypefilterGTD[0]!="XYZ":
    subframe=subframe[(subframe["weaptype1_txt"].isin(weaptypefilterGTD)) | (subframe["weaptype2_txt"].isin(weaptypefilterGTD))| (subframe["weaptype3_txt"].isin(weaptypefilterGTD))]
    sub_array=[]
    sub_array.append("for the weapontype(s):"+str(weaptypefilterGTD))
    criteria_txt_array.append(sub_array)
    criteria_array.extend(weaptypefilterGTD)

if targtypefilterGTD[0]!="XYZ":
    subframe=subframe[(subframe["targtype1_txt"].isin(targtypefilterGTD)) | (subframe["targtype2_txt"].isin(targtypefilterGTD))| (subframe["targtype3_txt"].isin(targtypefilterGTD))]
    sub_array=[]
    sub_array.append("for the targettype(s):"+str(targtypefilterGTD))
    criteria_txt_array.append(sub_array)
    criteria_array.extend(targtypefilterGTD)
criteria_txt_array=str(criteria_txt_array).replace("[","").replace("]","")
print(criteria_txt_array)
subframe=subframe.sort_values(by="Qdate", ascending=False)
subframe=subframe.reset_index(drop=True)

#then we split the dataframe according to the defined two-way comparison
if targtypeGTD_twowaycomparison[0]!="XYZ":
    comparison1=subframe[(subframe["targtype1_txt"]==targtypeGTD_twowaycomparison[0]) | (subframe["targtype2_txt"]==targtypeGTD_twowaycomparison[0])| (subframe["targtype3_txt"]==targtypeGTD_twowaycomparison[0])]
    comparison2=subframe[(subframe["targtype1_txt"]==targtypeGTD_twowaycomparison[1]) | (subframe["targtype2_txt"]==targtypeGTD_twowaycomparison[1])| (subframe["targtype3_txt"]==targtypeGTD_twowaycomparison[1])]
    comparisonarray=targtypeGTD_twowaycomparison
    twowaycomparison_German=targtypeGTD_twowaycomparison_German
    
if weaptypeGTD_twowaycomparison[0]!="XYZ":
    comparisonarray=weaptypeGTD_twowaycomparison
    comparison1=subframe[(subframe["weaptype1_txt"]==weaptypeGTD_twowaycomparison[0]) | (subframe["weaptype2_txt"]==weaptypeGTD_twowaycomparison[0])| (subframe["weaptype3_txt"]==weaptypeGTD_twowaycomparison[0])]
    comparison2=subframe[(subframe["weaptype1_txt"]==weaptypeGTD_twowaycomparison[1]) | (subframe["weaptype2_txt"]==weaptypeGTD_twowaycomparison[1])| (subframe["weaptype3_txt"]==weaptypeGTD_twowaycomparison[1])]
    comparisonarray=weaptypeGTD_twowaycomparison
    twowaycomparison_German=weaptypeGTD_twowaycomparison_German


if countryGTD_twowaycomparison[0]!="XYZ":
    comparison1=subframe[(subframe["country"]==countryGTD_twowaycomparison[0])]
    comparison2=subframe[(subframe["country"]==countryGTD_twowaycomparison[1])]
    comparisonarray=countryGTD_twowaycomparison
    twowaycomparison_German=countryGTD_twowaycomparison_German

if regionGTD_twowaycomparison[0]!="XYZ":
    comparison1=subframe[(subframe["region"]==regionGTD_twowaycomparison[0])]
    comparison2=subframe[(subframe["region"]==regionGTD_twowaycomparison[1])]
    comparisonarray=regionGTD_twowaycomparison
    twowaycomparison_German=regionGTD_twowaycomparison_German



comparison1=comparison1.sort_values(by="year").reset_index(drop=True).rename(columns={"deaths":twowaycomparison_German[0]})
comparison2=comparison2.sort_values(by="year").reset_index(drop=True).rename(columns={"deaths":twowaycomparison_German[1]})
#smallmultipledf1on1=pd.concat([comparison1[["year",comparisonarray[0]]],comparison2["year",comparisonarray[1]]],axis=1)
comparison1=comparison1[["year",twowaycomparison_German[0]]]
comparison1=comparison1.groupby(['year']).sum().reset_index()
comparison2=comparison2[["year",twowaycomparison_German[1]]]
comparison2=comparison2.groupby(['year']).sum().reset_index()

for year in range(1970, 2017):
#if we have no entry for one year between 1970 and 2017, we create one with two zeroes
    if (comparison1["year"]==year).any():
        pass
    else:
        comparison1=comparison1.append({"year":year,twowaycomparison_German[0]:0}, ignore_index=True)

comparison1=comparison1.sort_values(by="year").reset_index(drop=True)

for year in range(1970, 2017):
#if we have no entry for one year between 1970 and 2017, we create one with two zeroes
    if (comparison2["year"]==year).any():
        pass
    else:
        comparison2=comparison2.append({"year":year,twowaycomparison_German[1]:0}, ignore_index=True)

comparison2=comparison2.sort_values(by="year").reset_index(drop=True)





smallmultipledf1on1=pd.merge(comparison1, comparison2, left_on="year",right_on="year",left_index=False,  right_index=False,how='outer', sort=False)
smallmultipledf1on1=smallmultipledf1on1.sort_values(by="year")
smallmultipledf1on1=smallmultipledf1on1.fillna(0)
smallmultipledf1on1=smallmultipledf1on1.rename(index=str, columns={"year": "Jahr"})
# get a list of columns
cols = list(smallmultipledf1on1)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('Jahr')))

smallmultipledf1on1 = smallmultipledf1on1.loc[:, cols]
smallmultipledf1on1

smallmultipledf1on1.to_csv("ChartData/6TwoGroupComparison_custom.csv", index=False)
text_file = open("chartdata/6TwoGroupComparison_custom.txt", "w")
text_file.write("Untertitel: \n")
text_file.write("Die jährliche Zahl der Anschlagstoten für {0} und {1} im Vergleich im Kontext {2}".format(twowaycomparison_German[0], twowaycomparison_German[1], criteria_txt_array))
text_file.close()

print("DONE")

"for the weapontype(s):'Firearms'"
DONE


# <font color='blue'> Links to the charts </font>

## 6) Two-Way Lines / Grouped Bar Chart /Small Multiples (Custom)
Here, the data is prepared for a comparison between the two groups of interests. The same CSV can be used for 
* 6) [Lines Comparison Template](https://q.st.nzz.ch/item/9d7295744b026bebf19a2490f7740237)

<table><tr><td><img src='Screenshots/DoubleLineCustom_2.png' style="height: 400px;"></td></tr>
</table>
